In [1]:
local_dir = '/home/neilor/scielo_data/dados'
connect_cmd = "dbname='scielo_db' user='neilor' host='localhost' password='n1f2c3n1'"
%load_ext sql
%sql postgresql://neilor:n1f2c3n1@/scielo_db
import psycopg2 as pg

ModuleNotFoundError: No module named 'sql'

In [3]:
## create table com nomes de autores não qualificados como M ou F

In [6]:
query_create_nomes_nao_MouF = """
drop table if exists alacip.autores_nomes_nao_M_ou_F;
create table alacip.autores_nomes_nao_M_ou_F
(
    nome varchar,
    masc varchar,
    fem varchar
);

insert into alacip.autores_nomes_nao_M_ou_F
SELECT autor_nome_normal as nome, '' as masc, '' as fem
  FROM alacip.autores_selecionados
where autor_sexo like '#NONE'
and autor_nome not like ''
and position('.' in autor_nome) = 0
group by autor_nome_normal
order by autor_nome_normal
;
"""
    
%sql { query_create_nomes_nao_MouF }

Done.
Done.
1958 rows affected.


[]

In [7]:
import pandas as pd
import numpy as np

conn = pg.connect(connect_cmd)
df_doacoes = pd.read_sql('select * from alacip.autores_nomes_nao_M_ou_F', conn)
df_doacoes.fillna('#NULO')
planilha =  local_dir+'/autores_nomes_nao_M_ou_F_v2.xlsx'
df_doacoes.to_excel(planilha,sheet_name='Sheet1',header=True, index=False)
df_doacoes.head()
conn.close()
print("OK")

OK


In [5]:
conn = pg.connect(connect_cmd)
autores_df = pd.read_sql('select * from alacip.autores_selecionados',conn)
autores_df.head()
#autores_cursor.execute('sel')

#nome = 'alo neilor fermino camargo '.split()
#print(nome[0],nome[-1])

,artigo_scielo_id,autor_nome_completo,autor_sobrenome,autor_nome,autor_nome_normal,autor_sexo,autor_instituicao,autor_cidade_instituicao,autor_pq
0,S1415-47572007000600005,Flávio Vieira Meirelles,#NONE,Flávio,FLAVIO,M,Universidade de São Paulo,Pirassununga,#NONE
1,S0034-71402005000200006,Flávio Vilela Vieira,#NONE,Flávio,FLAVIO,M,Universidade Federal de Uberlândia,,#NONE
2,S1415-47572008000400010,Flávio V. Meirelles,#NONE,Flávio,FLAVIO,M,Universidade de São Paulo,Pirassununga,#NONE
3,S1415-47571999000400013,Flávio V. Meirelles,#NONE,Flávio,FLAVIO,M,USP,Ribeirão Preto,#NONE
4,S0034-71402007000400005,Francisco A. Alcaraz Garcia,#NONE,Francisco,FRANCISCO,M,Pontifícia Universidade Católica do Rio de Jan...,,#NONE


In [24]:
import pandas as pd
import numpy as np

def get_nome_completo_normal(nome_completo):
    if nome_completo != None:
        n0=nome_completo.upper()
        n1=n0.replace('.',' ')
        n2=n1.replace(' DO ',' ')
        n3=n2.replace(' DA ',' ')
        n4=n3.replace(' DE ',' ')
        n5=n4.replace(' NETO','-NETO')
        n6=n5.replace(' FILHO','-FILHO')
        n7=n6.replace(' JÚNIOR','-JUNIOR')
        n8=n7.replace(' JUNIOR','-JUNIOR')
        n9=n8.replace(' JR','-JR')
        nomes = n9.split()
        s=len(nomes)
        nomes_meio=[]
        for i in range(1,s-1):
            nomes_meio.append(nomes[i][0])

        nome_normal = nomes[0]
        for x in nomes_meio:
            nome_normal =nome_normal+' '+x
        nome_normal = nome_normal+' '+nomes[s-1]
        
        return(nome_normal)
    
def get_nome_sobrenome(nome_normal):
    n=nome_normal.spli()
    return n[0]+' '+n[-1]

df2=autores_df['autor_nome_completo'].map(get_nome_completo_normal)
df3=df2.value_counts().to_frame()



#df4=pd.concat([df2,df3],axis=1)

planilha =  local_dir+'/autores_producao.xlsx'
df3.to_excel(planilha,sheet_name='Sheet1',header=True, index=True)
df3

,autor_nome_completo
FAUSTO FORESTI,31
LUIS F MIGUEL,31
PAULO R ALMEIDA,30
CLAUDIO OLIVEIRA,28
IRACILDA SAMPAIO,20
PEDRO M GALETTI-JUNIOR,20
HORACIO SCHNEIDER,19
MARCOS A MACHADO,18
COSME D CRUZ,18
JOSÉ F S SARAIVA,18
